In [4]:
%pip install tweepy


Defaulting to user installation because normal site-packages is not writeable
  Using cached tweepy-4.14.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl.metadata (10 kB)
Using cached tweepy-4.14.0-py3-none-any.whl (98 kB)
Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
import tweepy
import pandas as pd
from textblob import TextBlob
import csv
import re
import string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import os
from dotenv import load_dotenv
nltk.download('vader_lexicon')  # Necesario solo una vez


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

<h1>authentication</h1>

In [7]:
# X (formerly Twitter) API credentials
load_dotenv()
BEARER_TOKEN = os.getenv("TWITTER_BEARER_TOKEN")
# Initialize Tweepy
client = tweepy.Client(BEARER_TOKEN)

<h1>Fetch & anali</h1>

In [8]:
# Function to fetch tweets
def fetch_tweets(query, max_results=10):
    tweets_data = client.search_recent_tweets(query=query, max_results=max_results, tweet_fields=['text'])
    tweets = []
    for tweet in tweets_data.data:

        tweet_text = clean_tweet(tweet.text)
        emotion= analyze_sentiment(tweet_text)
        tweets.append((tweet_text,emotion))
    return pd.DataFrame(tweets, columns=['Tweet', 'Emotion'])

<h1>Clean</h1>

In [9]:


def clean_tweet(text):
    # Elimina menciones (@usuario)
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    
    # Elimina hashtags (pero conserva la palabra)
    text = re.sub(r'#', '', text)
    
    # Elimina enlaces
    text = re.sub(r'http\S+|www.\S+', '', text)
    
    # Elimina emojis
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    # Elimina caracteres especiales y números
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Elimina espacios adicionales y convierte a minúsculas
    text = text.strip().lower()
    
    return text


<h1>Sentiment</h1>

In [10]:


sia = SentimentIntensityAnalyzer()

def analyze_sentiment(tweet_text):
    score = sia.polarity_scores(tweet_text)
    if score['compound'] >= 0.05:
        return "POS"
    elif score['compound'] <= -0.05:
        return "NEG"
    else:
        return "NEU"


<h1>main</h1>

In [11]:
df= fetch_tweets('(inseguridad OR robo OR atracos OR violencia OR delincuencia) Barranquilla -is:retweet', 10)
df.to_csv('prueba2.csv', index=False)